As a part of our paper review / response to reviewers, we are spot checking a sample of users in each account type (by opinion, bot status, qanon status), and getting all their tweets (max 25 or 50 should be plenty).

In this second approach, we'll concat the tweet text, to make user labeling easier. We can separate them with a double pipe or something like "tweet 1 text || tweet 2 text || tweet 3 text || etc."


# Setup

## Mounting the Drive

In [ ]:
import os

from google.colab import drive

drive.mount('/content/drive')
print(os.listdir(os.getcwd())) 

Mounted at /content/drive
['.config', 'drive', 'sample_data']


In [ ]:
# you might need to create a google drive SHORTCUT that has this same path
# ... or update the path to use your own google drive organization
DIRPATH = '/content/drive/MyDrive/Research/Disinfo Research Shared 2022'
print(DIRPATH)
os.path.isdir(DIRPATH)

/content/drive/MyDrive/Research/Disinfo Research Shared 2022


True

## Configuring Credentials 


In [ ]:
# google.cloud checks the file at path designated by the GOOGLE_APPLICATION_CREDENTIALS env var
# so we set it here using the shared credentials JSON file from our shared google drive
# and verify it for good measure

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(DIRPATH, "credentials", "tweet-research-shared-268bbccc0aac.json") 

GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS") # implicit check by google.cloud
print(GOOGLE_APPLICATION_CREDENTIALS) # verification for implicit check
print(os.path.isfile(GOOGLE_APPLICATION_CREDENTIALS)) # verification for implicit check

/content/drive/MyDrive/Research/Disinfo Research Shared 2022/credentials/tweet-research-shared-268bbccc0aac.json
True


# Helpers

### BigQuery Service

In [ ]:

from google.cloud import bigquery
from pandas import DataFrame

class BigQueryService():
    def __init__(self):
        self.client = bigquery.Client()

    def execute_query(self, sql, verbose=True):
        if verbose == True: 
            print(sql)
        job = self.client.query(sql)
        return job.result()
    
    def query_to_df(self, sql, verbose=True):
        """high-level wrapper to return a DataFrame"""
        results = self.execute_query(sql, verbose=verbose)
        records = [dict(row) for row in list(results)]
        df = DataFrame(records)
        return df


In [ ]:
bq_service = BigQueryService()
print(bq_service)

# Usage

### Users Sample

In [ ]:
USERS_PER_GROUP = 50

GROUPS = [
    # anti-trump
    {"opinion_community": 0, "is_bot": "false", "is_q": "false"},
    {"opinion_community": 0, "is_bot": "true",  "is_q": "false"},
    # pro-trump
    {"opinion_community": 1, "is_bot": "false", "is_q": "false"},
    {"opinion_community": 1, "is_bot": "true",  "is_q": "false"},
    # q-anon
    {"opinion_community": 1, "is_bot": "false", "is_q": "true"},
    {"opinion_community": 1, "is_bot": "true",  "is_q": "true"},
]

frames = []
for group in GROUPS:
    opinion_community = group["opinion_community"] #> 0
    is_bot = group["is_bot"] #> "false"
    is_q = group["is_q"] #> "false"
    print("ACCT TYPE:...", opinion_community, is_bot, is_q)

    sql = f"""
        SELECT 
            u.user_id
            ,u.opinion_community 
            ,u.is_bot 
            ,u.is_q 
            ,u.created_on 
            ,u.screen_name_count
            ,u.status_count
            ,u.rt_count
        FROM `tweet-research-shared.impeachment_2020.user_details_v20210806_slim` u 
        WHERE u.opinion_community={opinion_community} and u.is_bot={is_bot} and u.is_q={is_q}

        ORDER BY rand()
        LIMIT {USERS_PER_GROUP};
    """
    df = bq_service.query_to_df(sql, verbose=False)
    #print(df.head())
    frames.append(df)
    

ACCT TYPE:... 0 false false
ACCT TYPE:... 0 true false
ACCT TYPE:... 1 false false
ACCT TYPE:... 1 true false
ACCT TYPE:... 1 false true
ACCT TYPE:... 1 true true


In [ ]:
from pandas import concat

users_df = concat(frames)
print(len(users_df))
users_df.to_csv("users_sample_by_account_type_v2.csv")
users_df

300


,user_id,opinion_community,is_bot,is_q,created_on,screen_name_count,status_count,rt_count
0,14710012,0,False,False,2008-05-09,1,5,1
1,43295319,0,False,False,2009-05-29,1,1,1
2,239281184,0,False,False,2011-01-17,1,3,0
3,996168658077528065,0,False,False,2018-05-14,1,1,0
4,1073315597939822592,0,False,False,2018-12-13,1,1,1
...,...,...,...,...,...,...,...,...
45,585143406,1,True,True,2012-05-19,1,1024,1010
46,819615302715637760,1,True,True,2017-01-12,1,1205,1200
47,1203386270010900480,1,True,True,2019-12-07,3,785,738
48,1176824400760451079,1,True,True,2019-09-25,1,648,643


In [ ]:
print(users_df["opinion_community"].value_counts())
print(users_df["is_bot"].value_counts())
print(users_df["is_q"].value_counts())

1    200
0    100
Name: opinion_community, dtype: int64
False    150
True     150
Name: is_bot, dtype: int64
False    200
True     100
Name: is_q, dtype: int64


In [ ]:
print(users_df[["opinion_community", "is_bot", "is_q"]].value_counts())

opinion_community  is_bot  is_q 
0                  False   False    50
                   True    False    50
1                  False   False    50
                           True     50
                   True    False    50
                           True     50
dtype: int64


### Tweets from those Users

In [ ]:
user_ids = users_df["user_id"].tolist()
print(len(user_ids))
print(len(list(set(user_ids))))
print(user_ids[0:2])

300
300
[14710012, 43295319]


In [ ]:
MAX_TWEETS_PER_USER = 25

frames = []
for user_id in user_ids:

    sql = f"""
        SELECT 
            u.user_id
            ,u.created_on 
            ,u.screen_name_count
            ,u.screen_names
            ,u.status_count 
            ,u.rt_count 
            ,(u.rt_count / u.status_count) as rt_pct

            ,u.opinion_community 
            ,u.is_bot  
            ,u.is_q  

            ,up.descriptions as profile_descriptions

            ,string_agg(t.status_text, ' || ' ORDER BY rand() LIMIT {MAX_TWEETS_PER_USER}) as tweet_texts

        FROM `tweet-research-shared.impeachment_2020.tweets_v2` t
        JOIN `tweet-research-shared.impeachment_2020.user_details_v20210806_slim` u ON u.user_id = t.user_id
        LEFT JOIN `tweet-research-shared.impeachment_2020.user_profiles_v2` up on up.user_id = u.user_id
        WHERE t.user_id = {user_id}
        GROUP BY 1,2,3,4,5,6,7,8,9,10,11
    """
    df = bq_service.query_to_df(sql, verbose=False)
    #print(df.head())
    frames.append(df)
    print(len(frames))
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
user_tweets_df = concat(frames)
print(len(user_tweets_df))
user_tweets_df.to_csv("users_sample_by_account_type_v2_and_their_tweets.csv")
user_tweets_df

300


,user_id,created_on,screen_name_count,screen_names,status_count,rt_count,rt_pct,opinion_community,is_bot,is_q,profile_descriptions,tweet_texts
0,14710012,2008-05-09,1,TREYPOLE,5,1,0.200000,0,False,False,"ACTOR, MUSICIAN, FILMMAKER, DATABASE ARCHITECT...",RT @foxnewpolls: Daily FOX NEW POLL: (retweet ...
0,43295319,2009-05-29,1,ERICCRAIN,1,1,1.000000,0,False,False,DFS PRO. FORMER POKER PRO. STL SPORTS ENTHUSIA...,"RT @MattOswaltVA: ""I JUST GOT ARRESTED FOR MAK..."
0,239281184,2011-01-17,1,MY_BIRDIE98,3,0,0.000000,0,False,False,2 GIRLS 2 BOYS ARMY VET🇺🇲 BSN RN CCM CLNC BA P...,Just in time for impeachment!!! || @Ryan449955...
0,996168658077528065,2018-05-14,1,REPELTHEGOONS,1,0,0.000000,0,False,False,"PRO #2A, #REPELTHEGOONS. FOR GOD, COUNTRY & F...",I'd trust Casey Anthony with my kids more than...
0,1073315597939822592,2018-12-13,1,QMAN10251,1,1,1.000000,0,False,False,JUST LIVE MY LIFE NOT TALKING ANYTHING SERIOUS...,RT @jemelehill: Same NFL who thinks players kn...
...,...,...,...,...,...,...,...,...,...,...,...,...
0,585143406,2012-05-19,1,DENISBUCHAN,1024,1010,0.986328,1,True,True,CONSERVATIVE....LOVE TRUMP,RT @RudyGiuliani: “Giuliani voiced his support...
0,819615302715637760,2017-01-12,1,TINAHERIG,1205,1200,0.995851,1,True,True,"LOVE GOD, MY FAMILY, HAPPILY MARRIED, NOT A BO...",RT @IngrahamAngle: One of the senators should ...
0,1203386270010900480,2019-12-07,3,QPLUS45 | WRWYQPLUS | QPLUS_17,785,738,0.940127,1,True,True,FOR GOD AND COUNTRY 🙏🇺🇸🇺🇸🙏 #TRUMP2020 #MAGA #K...,RT @thebradfordfile: Nancy Pelosi is so bitter...
0,1176824400760451079,2019-09-25,1,JEFF25703425,648,643,0.992284,1,True,True,"PROUD DAD, PATRIOT, 2A. GARDENER. MASTER OF DU...",RT @ezridersos: No wonder @JoeBiden refuses to...
